# 🧮 5. Data Preprocessing
## 5.1 Tujuan

Tahap Data Preprocessing bertujuan untuk mengubah data audio mentah menjadi data numerik yang siap digunakan untuk proses modeling. Proses ini meliputi pembacaan file audio, normalisasi sinyal, penghapusan noise, pemotongan bagian diam, serta ekstraksi fitur statistik time series seperti mean, standard deviation, RMS, dan zero crossing rate (ZCR).

## 5.2 Langkah-Langkah Preprocessing
### a. Import Library yang Dibutuhkan

In [1]:
import os
import numpy as np
import librosa
import soundfile as sf       
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pickle

## b.Lakukan preprocessing

membuat fungsi preprocessing audio, yang akan:

✅ Membaca semua file .wav dari masing-masing orang dan kelas

✅ Normalisasi amplitudo

✅ Trimming bagian diam

✅ Zero-padding agar durasi seragam (misalnya 2 detik @16kHz = 32000 sampel)

✅ Simpan hasil bersih ke folder preprocces_person1,2

In [2]:
import os
import librosa
import numpy as np
from tqdm import tqdm

# ==========================
# PATH DATASET
# ==========================
DATASET_PERSON1 = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Audio_recognition\Dataset_Voice_pertama"
DATASET_PERSON2 = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Audio_recognition\Dataset_Voice_kedua"

# ==========================
# PARAMETER PREPROCESSING
# ==========================
TARGET_SR = 16000          
TARGET_DURATION = 3.0      
TARGET_LEN = int(TARGET_SR * TARGET_DURATION)

# ==========================
# FUNGSI PREPROCESSING
# ==========================
def preprocess_audio(file_path):
    try:
        y, sr = librosa.load(file_path, sr=TARGET_SR)
        y = librosa.util.normalize(y)
        y, _ = librosa.effects.trim(y, top_db=25)

        # Balancing durasi
        if len(y) < TARGET_LEN:
            y = np.pad(y, (0, TARGET_LEN - len(y)), mode='constant')
        else:
            y = y[:TARGET_LEN]

        return y
    except Exception as e:
        print(f"❌ Error memproses {file_path}: {e}")
        return None

# ==========================
# PREPROCESS DATASET PERSON 1
# ==========================
print("\n🎤 Memproses: Dataset_Voice_pertama")
data_person1 = {}  # dictionary: {'buka': [array, ...], 'tutup': [array, ...]}

valid_folders = [f for f in os.listdir(DATASET_PERSON1) if f.endswith("_wav")]

for label_folder in valid_folders:
    label_path = os.path.join(DATASET_PERSON1, label_folder)
    if not os.path.isdir(label_path):
        continue

    label_clean = label_folder.replace("_wav", "")
    data_person1[label_clean] = []

    for file in tqdm(os.listdir(label_path), desc=f"Person1/{label_clean}"):
        if file.endswith(".wav"):
            in_path = os.path.join(label_path, file)
            y = preprocess_audio(in_path)
            if y is not None:
                data_person1[label_clean].append(y)

# ==========================
# PREPROCESS DATASET PERSON 2
# ==========================
print("\n🎙️ Memproses: Dataset_Voice_kedua")
data_person2 = {}  # dictionary: {'buka': [array, ...], 'tutup': [array, ...]}

for label_folder in os.listdir(DATASET_PERSON2):
    label_path = os.path.join(DATASET_PERSON2, label_folder)
    if not os.path.isdir(label_path):
        continue

    data_person2[label_folder] = []

    for file in tqdm(os.listdir(label_path), desc=f"Person2/{label_folder}"):
        if file.endswith(".wav"):
            in_path = os.path.join(label_path, file)
            y = preprocess_audio(in_path)
            if y is not None:
                data_person2[label_folder].append(y)

print("\n✅ Preprocessing selesai.")



🎤 Memproses: Dataset_Voice_pertama


Person1/Tutup: 100%|██████████| 50/50 [00:00<00:00, 50.49it/s]



🎙️ Memproses: Dataset_Voice_kedua


Person2/Tutup: 100%|██████████| 48/48 [00:00<00:00, 50.58it/s]


✅ Preprocessing selesai.


### c. Fungsi Ekstraksi Fitur Statistik

Fungsi ini akan membaca setiap file .wav, melakukan preprocessing dasar, lalu menghitung fitur statistik dari sinyal time domain.

In [3]:

SAMPLE_RATE = 16000

def extract_features(y, sr=SAMPLE_RATE):
    """
    Ekstraksi fitur dari sinyal audio y (numpy array) dengan sample rate sr.
    Output: list fitur
    """

    features = []

    # -------------------
    # 1. Statistik dasar
    # -------------------
    features += [
        np.mean(y),
        np.std(y),
        np.var(y),
        np.mean((y - np.mean(y))**3)/(np.std(y)**3 + 1e-6),  # skewness
        np.mean((y - np.mean(y))**4)/(np.std(y)**4 + 1e-6),  # kurtosis
        np.sqrt(np.mean(y**2)),  # RMS global
        np.mean(librosa.feature.zero_crossing_rate(y)),
        np.std(librosa.feature.zero_crossing_rate(y)),
        np.max(y) - np.min(y)
    ]

    # -------------------
    # 2. Spektral
    # -------------------
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    spec_flatness = librosa.feature.spectral_flatness(y=y)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)

    features += [
        np.mean(spec_centroid), np.std(spec_centroid),
        np.mean(spec_bandwidth), np.std(spec_bandwidth),
        np.mean(spec_contrast), np.std(spec_contrast),
        np.mean(spec_rolloff), np.std(spec_rolloff),
        np.mean(spec_flatness), np.std(spec_flatness),
        np.mean(chroma), np.std(chroma)
    ]

    # MFCC 13 koef
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    for i in range(13):
        features.append(np.mean(mfcc[i]))
    for i in range(13):
        features.append(np.std(mfcc[i]))

    # Delta MFCC
    mfcc_delta = librosa.feature.delta(mfcc)
    for i in range(13):
        features.append(np.mean(mfcc_delta[i]))
    for i in range(13):
        features.append(np.std(mfcc_delta[i]))

    # -------------------
    # 3. Temporal / Energy
    # -------------------
    # RMS per frame
    rms = librosa.feature.rms(y=y)
    features.append(np.mean(rms))
    features.append(np.std(rms))

    # Onset envelope
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    features.append(np.mean(onset_env))
    features.append(np.std(onset_env))

    # Tempo
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    features.append(float(tempo[0]))

    # Autocorrelation (lag of max peak)
    autocorr = np.correlate(y, y, mode='full')
    mid = len(autocorr)//2
    features.append(np.argmax(autocorr[mid+1:]) + 1)

    return features


### d. Looping untuk Mengambil Fitur dari Semua File

Kita akan mengambil semua file dari folder buka dan tutup, lalu menambahkan label untuk tiap file.

In [4]:
import pandas as pd

data_rows = []
labels = []

# Gabungkan semua data preprocessing di memory
all_data = [
    ("person1", data_person1),
    ("person2", data_person2)
]

for person_tag, data_dict in all_data:
    for label, audio_list in data_dict.items():
        for y_audio in audio_list:
            feats = extract_features(y_audio)  # array y langsung
            data_rows.append(feats)
            labels.append(f"{person_tag}_{label}")  # misal: person1_buka

# ==========================
# Buat nama kolom sesuai fitur baru
# ==========================
columns = []

# Statistik dasar
columns += ["mean","std","var","skew","kurtosis","rms_global","zcr_mean","zcr_std","amplitude_range"]

# Spektral
columns += [
    "spec_centroid_mean","spec_centroid_std",
    "spec_bandwidth_mean","spec_bandwidth_std",
    "spec_contrast_mean","spec_contrast_std",
    "spec_rolloff_mean","spec_rolloff_std",
    "spec_flatness_mean","spec_flatness_std",
    "chroma_mean","chroma_std"
]

# MFCC 13 + delta MFCC 13
for i in range(1,14):
    columns.append(f"mfcc{i}_mean")
for i in range(1,14):
    columns.append(f"mfcc{i}_std")
for i in range(1,14):
    columns.append(f"mfcc{i}_delta_mean")
for i in range(1,14):
    columns.append(f"mfcc{i}_delta_std")

# Temporal / Energy
columns += ["rms_mean","rms_std","onset_mean","onset_std","tempo","autocorr_lag"]

# ==========================
# Simpan ke CSV
# ==========================
df = pd.DataFrame(data_rows, columns=columns)
df['label'] = labels
OUTPUT_CSV = r"D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Audio_recognition\features_audio.csv"
df.to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ Fitur disimpan ke {OUTPUT_CSV}")


C:\Users\user\AppData\Local\Temp\ipykernel_3748\2807756471.py:73: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)



✅ Fitur disimpan ke D:\KULIAH\SEMESTER 5\Program Saint Data\Uranus\myfirstbook\Audio_recognition\features_audio.csv


### e. Analisa dan pemilihan Fitur terbaik

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def feature_importance_analysis(X_train, y_train, X_val, y_val, cumulative_threshold=0.95):
    """
    Analisis feature importance menggunakan Random Forest.
    - cumulative_threshold: jumlah cumulative importance untuk memilih fitur terbaik (default 95%)
    """
    print("Training Random Forest untuk analisis feature importance...")
    rf_temp = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
    rf_temp.fit(X_train, y_train)

    # Feature importance
    importances = pd.Series(rf_temp.feature_importances_, index=X_train.columns).sort_values(ascending=False)

    # Cumulative importance
    cumulative = importances.cumsum()
    selected_features = cumulative[cumulative <= cumulative_threshold].index.tolist()
    # Pastikan selalu setidaknya 1 fitur
    if len(selected_features) == 0:
        selected_features = [importances.idxmax()]

    removed_features = [f for f in X_train.columns if f not in selected_features]

    # Visualisasi cumulative importance
    plt.figure(figsize=(12,6))
    plt.plot(np.arange(len(importances))+1, cumulative.values, marker='o')
    plt.axhline(y=cumulative_threshold, color='red', linestyle='--', label=f'Cumulative threshold={cumulative_threshold}')
    plt.xlabel('Fitur (urut descending importance)')
    plt.ylabel('Cumulative Importance')
    plt.title('Cumulative Feature Importance')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"\nHASIL ANALISIS FEATURE IMPORTANCE")
    print(f"{'-'*50}")
    print(f"Total fitur awal: {len(importances)}")
    print(f"Fitur yang dipilih: {len(selected_features)}")
    print(f"Fitur yang dihapus: {len(removed_features)}")

    # Dataset terpilih
    X_train_selected = X_train[selected_features].copy()
    X_val_selected = X_val[selected_features].copy()

    return X_train_selected, X_val_selected, selected_features, removed_features, importances


## Deteksi Outlier dan penanganannya

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# ===============================
# Fungsi deteksi outlier dengan beberapa metode
# ===============================
def detect_outliers_multiple_methods(df, contamination=0.05, scale=True):
    """
    Deteksi outlier menggunakan 3 metode:
    1. IQR
    2. Z-Score
    3. Isolation Forest
    """
    X = df.drop(columns=['label'])
    
    # Optional: standarisasi
    if scale:
        scaler = StandardScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    else:
        X_scaled = X.copy()
    
    # 1. IQR (gunakan data asli)
    Q1 = X.quantile(0.25)
    Q3 = X.quantile(0.75)
    IQR = Q3 - Q1
    outlier_mask_iqr = ((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)
    
    # 2. Z-Score
    outlier_mask_zscore = (np.abs(X_scaled) > 3).any(axis=1)
    
    # 3. Isolation Forest
    iso_forest = IsolationForest(contamination=contamination, random_state=42)
    outlier_mask_iso = iso_forest.fit_predict(X_scaled) == -1
    
    return {
        'iqr_outliers': outlier_mask_iqr,
        'zscore_outliers': outlier_mask_zscore,
        'isolation_outliers': outlier_mask_iso,
        'X_scaled': X_scaled
    }

# ===============================
# Analisis dan visualisasi outlier
# ===============================
def analyze_outliers(df, contamination=0.05, scale=True):
    """
    Analisis outlier per kelas & metode
    """
    print("ANALISIS OUTLIER")
    print("="*50)
    
    outliers = detect_outliers_multiple_methods(df, contamination=contamination, scale=scale)
    
    # Hitung jumlah outlier
    def print_counts(out_dict):
        for method in ['iqr_outliers','zscore_outliers','isolation_outliers']:
            count = out_dict[method].sum()
            print(f"{method.replace('_outliers','').capitalize()}: {count} ({count/len(df)*100:.1f}%)")
    
    print_counts(outliers)
    
    # Visualisasi bar chart per kelas
    summary = df.groupby('label').apply(
        lambda x: pd.Series({
            'IQR': detect_outliers_multiple_methods(x, contamination=contamination, scale=scale)['iqr_outliers'].sum(),
            'Z-Score': detect_outliers_multiple_methods(x, contamination=contamination, scale=scale)['zscore_outliers'].sum(),
            'Isolation': detect_outliers_multiple_methods(x, contamination=contamination, scale=scale)['isolation_outliers'].sum()
        })
    ).reset_index()
    
    melted = pd.melt(summary, id_vars=['label'], var_name='Method', value_name='Count')
    plt.figure(figsize=(10,6))
    sns.barplot(data=melted, x='label', y='Count', hue='Method')
    plt.title('Outliers per Class & Method')
    plt.show()
    
    return outliers

# ===============================
# Hapus outlier dengan consensus minimal 2 metode
# ===============================
def remove_outliers_consensus(df, outlier_results, min_methods=2):
    """
    Menghapus data yang dianggap outlier oleh minimal min_methods metode
    """
    outlier_counts = (
        outlier_results['iqr_outliers'].astype(int) +
        outlier_results['zscore_outliers'].astype(int) +
        outlier_results['isolation_outliers'].astype(int)
    )
    consensus_outliers = outlier_counts >= min_methods
    clean_df = df[~consensus_outliers].copy()
    removed_df = df[consensus_outliers].copy()
    
    print(f"\nOutlier removal dengan consensus >= {min_methods} metode:")
    print(f"  Original samples: {len(df)}")
    print(f"  Outliers removed: {len(removed_df)} ({len(removed_df)/len(df)*100:.1f}%)")
    print(f"  Clean samples: {len(clean_df)} ({len(clean_df)/len(df)*100:.1f}%)")
    
    print("\nDistribusi kelas:")
    print(f"  Original: {dict(df['label'].value_counts())}")
    print(f"  Clean: {dict(clean_df['label'].value_counts())}")
    print(f"  Removed: {dict(removed_df['label'].value_counts())}")
    
    return clean_df, removed_df

# ===============================
# Contoh eksekusi
# ===============================
# Misal train_selected dan val_selected sudah ada dari feature selection
# train_outliers = analyze_outliers(train_selected, contamination=0.05)
# train_clean, train_removed = remove_outliers_consensus(train_selected, train_outliers, min_methods=2)
# val_outliers = analyze_outliers(val_selected, contamination=0.05)
# val_clean, val_removed = remove_outliers_consensus(val_selected, val_outliers, min_methods=2)

# ===============================
# Simpan hasil ke CSV
# ===============================
# train_clean.to_csv("train_features_clean.csv", index=False)
# val_clean.to_csv("val_features_clean.csv", index=False)
# train_removed.to_csv("train_outliers_removed.csv", index=False)
# val_removed.to_csv("val_outliers_removed.csv", index=False)
